# Филтрирање на слика во фреквенциски домен

## Теорија

За да филтрираме слика во фреквенциски домен 
потребно е да ја помножиме сликата во фреквенциски 
домен со филтерот (кернел) кој е исто така во фреквенциски домен.

### Равенка за филтрирање на слика во фреквенциски домен: 

$$I_{f,filt} = HI_f$$

За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:

### Bandpass гаусов филтер:

$$H(i,j) = e^\frac{-d(i,j)^2} {f_h^2}\left(1 - e^\frac{-d(i,j)^2} {f_l^2}\right)$$

Каде што $d(i,j)$ е евклидовото растојанието на даден пиксел од центарот на сликата, додека $f_h$ и $f_l$ се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме.
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација. Фуриевата трансформација е дефинирана како:

### Фуриева трансформација во апсолутно-непрекинат домен:

$$F(u,v) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} f(x,y) e^{-i2\pi(ux+vy)} dxdy$$

А во дискретниот домен горната равенка може да биде преведена како

### Фуриева трансформација во дискретен домен:

$$F(u,v) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {-i2\pi(umx_0+vny_0)}$$
        
Конверзија на 2D сликата од фреквенциски во просторен домен се прави со
користење на инверзна Фуриева трансформација. Инверзната Фуриева
трансформација е дефинирана како:

### Инверзни равенки:

$$ f(x,y)= \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} F(u,v) e^{i2\pi(ux+vy)} dudv$$

А во дискретниот домен горната равенка може да биде преведена како

$$F(x,y) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {i2\pi(xmu_0+ynv_0)}$$

In [7]:
%%file create_filter.m

function filter = create_filter(nx,ny,d0,d1)
    filter = ones(nx,ny);
    for i = 0:nx-1
        for j = 0:ny-1
            dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
            filter(i+1,j+1) = exp(-(dist^2)/(d1^2)).*(1.0-exp(-(dist^2)/(d0^2)));
        end
    end
    filter(nx/2+1,ny/2+1)=1;
end

Created file 'C:\Users\atana\Desktop\miikt\create_filter.m'.


In [3]:
clear all
%read image
im = double(imread('Barbara.tif'));
%size of the image
[nx ny] = size(im);
%Transform the image from spatial to frequency domain
im_fft = fftshift(fft2(im));
% Create Gaussian bandpass kernels, in frequency domain, for 20 different cut off frequencies and filter the image
p=1;
for i = 1:10:101
q=1;
    for j = 1:10:101
        %Create the bandpass kernel
        tmp1 = create_filter(nx,ny,i,j);
        %Filter the image with the bandpass kernel and transform it back to spatial domain
        tmp = mat2gray(abs(ifft2(ifftshift(tmp1.*im_fft))));
        %Save the filtered image (in spatial domain) and kernel (in frequency domain) for visualization 
        filtered_image(p,q,:) = tmp(:);
        filter_kernel(p,q,:) = tmp1(:);
        q=q+1;
    end
    p=p+1;
end
%zachuvuvanje na matrici na slikata i kernelot vo iminjata.mat
save filtered_image 
save filter_kernel

In [1]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

%matplotlib nbagg

amp = widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=10,
    step=1,
    description='Frequency cutoffs:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout={'width': '100%'}
)

display(amp)

IntRangeSlider(value=(0, 10), continuous_update=False, description='Frequency cutoffs:', layout=Layout(width='…

In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import base64


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


image_filename = 'C:\\Users\\atana\\Downloads\\Barbara.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.Div("Frequency cutoffs:"),
    dcc.RangeSlider(
        id='my-range-slider',
        min=0,
        max=10,
        step=0.5,
        value=[1, 4]
    ),
    html.Div(id='output-container-range-slider'),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
])


@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('my-range-slider', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)


if __name__ == '__main__':
    app.run_server(port=8009,
                   debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8009/ (Press CTRL+C to quit)


In [9]:
import scipy.io as sio
filtered_image = sio.loadmat('filtered_image.mat')
filter_kernel = sio.loadmat('filter_kernel.mat')
filt_images = filtered_image['filtered_image']
filt_kernels = filter_kernel['filter_kernel']
temp = filt_kernels[0][5][0:]
temp = temp.reshape(402,566)
temp = np.rot90(temp)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imgplot = plt.imshow(temp, cmap = "gray")




NameError: name 'np' is not defined